In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from dataset import *
from model import *
import torch.nn.functional as F
from tqdm import tqdm
from util import *
from train import train_eval
from torchsummary import summary
from torcheeg.models import *
from torcheeg.transforms import RandomFrequencyShift
from torcheeg import trainers
import scipy.signal as sig
from torch.utils.data import Dataset
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import numpy as np
import os
from scipy import signal
import pywt


In [ ]:
data_path = '/content/drive/MyDrive/C247/project/dataset'
device = "cuda" if torch.cuda.is_available() else "cpu"

# Dataset preparation & Preprocessing

In [ ]:
train_dataset = EEGDataset(data_path=data_path)
val_dataset = EEGDataset(data_path, mode='val')
test_dataset = EEGDataset(data_path=data_path, mode='test')

train_loader  = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


# Plotting EEG Data for subjects

In [ ]:
import matplotlib.pyplot as plt

train_dataset = EEGDataset(data_path=data_path,subject=4)

data = train_dataset.X  ## Note that data.shape = [235,22,500]
data = data.numpy()
data = smooth_data(data,ws=5)
y = train_dataset.y.numpy()
class_averages = []

for class_label in range(4):
    data_class = data[y == class_label]
    class_average = np.mean(data_class, axis=1)
    class_averages.append(class_average)

fig, axs = plt.subplots(2, 1, figsize=(10, 10))
for class_label, class_average in enumerate(class_averages):
    axs[0].plot(np.mean(class_average,axis=0), label=f'Class {class_label}')

axs[0].set_title('EEG data- Subject 5')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Signal magnitude')
axs[0].legend()

#####################################################

train_dataset = EEGDataset(data_path=data_path,subject=8)

data = train_dataset.X
data = data.numpy()
y = train_dataset.y.numpy()
class_averages = []
data = smooth_data(data,ws=5)
for class_label in range(4):
    data_class = data[y == class_label]
    class_average = np.mean(data_class, axis=1)
    class_averages.append(class_average)

plt.figure(figsize=(10, 6))

for class_label, class_average in enumerate(class_averages):
    axs[1].plot(np.mean(class_average,axis=0), label=f'Class {class_label}')

axs[1].set_title('EEG data- Subject 9')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Signal magnitude')
axs[1].legend()
plt.savefig('subjectplot.png')
plt.show()

# Train & Test model

In [ ]:
# Initialize network, you can use either of the models

model = EEGNet(chunk_size=500, num_electrodes=22, num_classes=4).to(device=device)
# model = LSTM(num_electrodes=22, num_classes=4).to(device=device)
# model = CNNLSTM(cnn_input_size = 22, rnn_input_size =40, hidden_size=50,  output_dim=4, dropout=0.5).to(device=device)
# model = DCNN_elu(n_output=4).to(device=device)
# model = TSCeption( num_electrodes=22, num_classes=4).to(device=device)

# Hyperparamaters
learning_rate = 1e-4
batch_size = 64
epochs = 150

# Loss + optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
trainer = trainers.ClassifierTrainer(model=model,
                                num_classes=4,
                                lr=1e-4,
                                weight_decay=1e-4,
                                accelerator="gpu")

In [ ]:
trainer.fit(train_loader,
            val_loader,
            max_epochs=125,
            default_root_dir=f'./checkpoints/ATCNet/',
            enable_progress_bar=True,
            limit_val_batches=0.0)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type             | Params
---------------------------------------------------
0 | model         | EEGNet           | 2.2 K 
1 | ce_fn         | CrossEntropyLoss | 0     
2 | train_loss    | MeanMetric       | 0     
3 | val_loss      | MeanMetric       | 0     
4 | test_loss     | MeanMetric       | 0     
5 | train_metrics | MetricCollection | 0     
6 | val_metrics   | MetricCollection | 0     
7 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
2.2 K     Trainable params


Training: |          | 0/? [00:00<?, ?it/s]

[2024-03-13 06:28:38] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.447 train_accuracy: 0.828 

INFO:torcheeg:
[Train] train_loss: 0.447 train_accuracy: 0.828 

[2024-03-13 06:28:40] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.448 train_accuracy: 0.826 

INFO:torcheeg:
[Train] train_loss: 0.448 train_accuracy: 0.826 

[2024-03-13 06:28:41] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.429 train_accuracy: 0.835 

INFO:torcheeg:
[Train] train_loss: 0.429 train_accuracy: 0.835 

[2024-03-13 06:28:43] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.449 train_accuracy: 0.829 

INFO:torcheeg:
[Train] train_loss: 0.449 train_accuracy: 0.829 

[2024-03-13 06:28:45] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.443 train_accuracy: 0.827 

INFO:torcheeg:
[Train] train_loss: 0.443 train_accuracy: 0.827 

[2024-03-13 06:28:46] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.438 train_accuracy: 0.828 

INFO:torcheeg:
[Train] train_loss: 0.438 train_accuracy: 0.828 

[2024-03-1

In [ ]:
score = trainer.test(test_loader,
                    enable_progress_bar=True,
                    enable_model_summary=True)[0]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[2024-03-13 06:31:07] INFO (torcheeg/MainThread) 
[Test] test_loss: 0.709 test_accuracy: 0.743 

INFO:torcheeg:
[Test] test_loss: 0.709 test_accuracy: 0.743 



┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7426636815071106     │
│         test_loss         │    0.7093983888626099     │
└───────────────────────────┴───────────────────────────┘